# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 29 2022 2:43PM,247587,10,EYE42986BIS,Eye Pharma Inc,Released
1,Sep 29 2022 2:40PM,247586,19,ACG-2102491408,ACG North America LLC,Released
2,Sep 29 2022 2:22PM,247396,21,OTM100945533,"NBTY Global, Inc.",Released
3,Sep 29 2022 2:22PM,247585,10,SOTAH0000794,"Nextsource Biotechnology, LLC",Released
4,Sep 29 2022 2:22PM,247585,10,SONSB0001959,"Nextsource Biotechnology, LLC",Released
5,Sep 29 2022 2:22PM,247585,10,SONSB0001961,"Nextsource Biotechnology, LLC",Executing
6,Sep 29 2022 2:22PM,247585,10,SONSB0001960,"Nextsource Biotechnology, LLC",Executing
7,Sep 29 2022 2:06PM,247584,10,PRF-38219,Bio-PRF,Released
8,Sep 29 2022 2:06PM,247584,10,PRF-38227,Bio-PRF,Released
9,Sep 29 2022 1:58PM,247581,10,8583921,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,247584,Released,2
17,247585,Executing,2
18,247585,Released,2
19,247586,Released,1
20,247587,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247581,NaN,NaN,1.0
247584,NaN,NaN,2.0
247585,NaN,2.0,2.0
247586,NaN,NaN,1.0
247587,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0.0,0.0,1.0
247527,11.0,0.0,5.0
247534,0.0,0.0,1.0
247542,0.0,40.0,1.0
247552,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0,0,1
247527,11,0,5
247534,0,0,1
247542,0,40,1
247552,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,0,0,1
1,247527,11,0,5
2,247534,0,0,1
3,247542,0,40,1
4,247552,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,,,1
1,247527,11,,5
2,247534,,,1
3,247542,,40,1
4,247552,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 29 2022 2:43PM,247587,10,Eye Pharma Inc
1,Sep 29 2022 2:40PM,247586,19,ACG North America LLC
2,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc."
3,Sep 29 2022 2:22PM,247585,10,"Nextsource Biotechnology, LLC"
7,Sep 29 2022 2:06PM,247584,10,Bio-PRF
9,Sep 29 2022 1:58PM,247581,10,Eywa Pharma Inc.
10,Sep 29 2022 1:52PM,247580,10,ISDIN Corporation
11,Sep 29 2022 1:43PM,247578,10,ISDIN Corporation
22,Sep 29 2022 1:43PM,247576,10,ISDIN Corporation
53,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 29 2022 2:43PM,247587,10,Eye Pharma Inc,,,1
1,Sep 29 2022 2:40PM,247586,19,ACG North America LLC,,,1
2,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc.",,,1
3,Sep 29 2022 2:22PM,247585,10,"Nextsource Biotechnology, LLC",,2,2
4,Sep 29 2022 2:06PM,247584,10,Bio-PRF,,,2
5,Sep 29 2022 1:58PM,247581,10,Eywa Pharma Inc.,,,1
6,Sep 29 2022 1:52PM,247580,10,ISDIN Corporation,,1,
7,Sep 29 2022 1:43PM,247578,10,ISDIN Corporation,,1,10
8,Sep 29 2022 1:43PM,247576,10,ISDIN Corporation,,,31
9,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc.",,40,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 2:43PM,247587,10,Eye Pharma Inc,1,,
1,Sep 29 2022 2:40PM,247586,19,ACG North America LLC,1,,
2,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc.",1,,
3,Sep 29 2022 2:22PM,247585,10,"Nextsource Biotechnology, LLC",2,2,
4,Sep 29 2022 2:06PM,247584,10,Bio-PRF,2,,
5,Sep 29 2022 1:58PM,247581,10,Eywa Pharma Inc.,1,,
6,Sep 29 2022 1:52PM,247580,10,ISDIN Corporation,,1,
7,Sep 29 2022 1:43PM,247578,10,ISDIN Corporation,10,1,
8,Sep 29 2022 1:43PM,247576,10,ISDIN Corporation,31,,
9,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc.",1,40,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 2:43PM,247587,10,Eye Pharma Inc,1,,
1,Sep 29 2022 2:40PM,247586,19,ACG North America LLC,1,,
2,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc.",1,,
3,Sep 29 2022 2:22PM,247585,10,"Nextsource Biotechnology, LLC",2,2,
4,Sep 29 2022 2:06PM,247584,10,Bio-PRF,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 2:43PM,247587,10,Eye Pharma Inc,1.0,NaN,NaN
1,Sep 29 2022 2:40PM,247586,19,ACG North America LLC,1.0,NaN,NaN
2,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Sep 29 2022 2:22PM,247585,10,"Nextsource Biotechnology, LLC",2.0,2.0,NaN
4,Sep 29 2022 2:06PM,247584,10,Bio-PRF,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 2:43PM,247587,10,Eye Pharma Inc,1.0,0.0,0.0
1,Sep 29 2022 2:40PM,247586,19,ACG North America LLC,1.0,0.0,0.0
2,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Sep 29 2022 2:22PM,247585,10,"Nextsource Biotechnology, LLC",2.0,2.0,0.0
4,Sep 29 2022 2:06PM,247584,10,Bio-PRF,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2228157,58.0,24.0,11.0
15,247542,1.0,40.0,0.0
16,742644,4.0,2.0,0.0
19,247586,1.0,0.0,0.0
21,247396,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2228157,58.0,24.0,11.0
1,15,247542,1.0,40.0,0.0
2,16,742644,4.0,2.0,0.0
3,19,247586,1.0,0.0,0.0
4,21,247396,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,58.0,24.0,11.0
1,15,1.0,40.0,0.0
2,16,4.0,2.0,0.0
3,19,1.0,0.0,0.0
4,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,15,Released,1.0
2,16,Released,4.0
3,19,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,11.0,0.0,0.0,0.0,0.0
Executing,24.0,40.0,2.0,0.0,0.0
Released,58.0,1.0,4.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,11.0,0.0,0.0,0.0,0.0
1,Executing,24.0,40.0,2.0,0.0,0.0
2,Released,58.0,1.0,4.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,11.0,0.0,0.0,0.0,0.0
1,Executing,24.0,40.0,2.0,0.0,0.0
2,Released,58.0,1.0,4.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()